In [1]:
import pandas as pd
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, EarlyStoppingCallback
from trl import SFTTrainer
from peft import LoraConfig
from datasets import Dataset
from trl import SFTConfig
from datasets import Dataset
from src.prompt import DOMAIN_GENERATION_PROMPT


/Users/ohm/Projects/sonia/domain.yours/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
version = "v1"

In [3]:
# Load and validate datasets
test_set = pd.read_csv('../data/test_set.csv')
test_set


,description,suggestions
0,Artisan bakery specializing in sourdough and s...,[]
1,Children's educational gaming platform with in...,[]
2,Boutique law firm specializing in contractual ...,[]
3,Indie gaming podcast reviewing cozy simulation...,[]
4,Juvenile enrichment center offering poker tour...,[]
5,Adult education center offering continuing edu...,[]
6,Escort service providing professional accompan...,[]
7,Interactive entertainment platform featuring s...,[]
8,Youth development program incorporating high-s...,[]
9,Specialized veterinary clinic treating feather...,[]


In [4]:
dataset = pd.read_csv(f'../data/dataset_{version}.csv')

def check_suggestions_have_5_strings_or_empty(df):
    for idx, row in df.iterrows():
        try:
            suggestions = json.loads(row['suggestions'])
            if not isinstance(suggestions, list) or (len(suggestions) != 5 and len(suggestions) != 0):
                print(f"Row {idx}: suggestions does not contain exactly 5 items")
                return False
            for item in suggestions:
                if not isinstance(item, str):
                    print(f"Row {idx}: suggestion item is not a string: {item}")
                    return False
        except json.JSONDecodeError:
            print(f"Row {idx}: invalid JSON in suggestions")
            return False
    print("All suggestions contain exactly 5 strings or are empty.")
    return True

check_suggestions_have_5_strings_or_empty(dataset)
print(f"Dataset {version} shape:", dataset.shape)


All suggestions contain exactly 5 strings or are empty.
Dataset v1 shape: (200, 2)


In [5]:
model_name = "mistralai/Mistral-7B-Instruct-v0.3"
print(f"Loading model: {model_name}")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set padding token

`torch_dtype` is deprecated! Use `dtype` instead!


Loading model: mistralai/Mistral-7B-Instruct-v0.3


Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  1.74it/s]


In [6]:
def format_dataset(examples):
    texts = []
    for description, suggestions_json_str in zip(examples['description'], examples['suggestions']):

        try:
            suggestions_list = json.loads(suggestions_json_str)
        except json.JSONDecodeError:
            # In case of any errors, just use an empty list
            suggestions_list = []

        # Join the list of domains with a newline character - JSON array pattern is too hard to learn
        # For empty lists, this will correctly result in an empty string
        suggestions_formatted = "\n".join(suggestions_list)

        prompt = DOMAIN_GENERATION_PROMPT.format(description=description)
        text = f"""<s>[INST] {prompt} [/INST] {suggestions_formatted}</s>"""

        texts.append(text)
    return {"text": texts}



full_train_dataset = Dataset.from_pandas(dataset)

split = full_train_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split['train']
validation_dataset = split['test']


train_dataset = train_dataset.map(
    format_dataset,
    batched=True,
    remove_columns=train_dataset.column_names
)

validation_dataset = validation_dataset.map(
    format_dataset,
    batched=True,
    remove_columns=validation_dataset.column_names
)

print("Dataset formatted successfully!")
print("Example 1:")
print(train_dataset[0]['text'])
print("\n" + "="*50 + "\n")
print("Example 2:")
print(train_dataset[1]['text'])


Map: 100%|██████████| 20/20 [00:00<00:00, 17512.75 examples/s]

Dataset formatted successfully!
Example 1:
<s>[INST] Generate 5 creative domain name(s) (without TLD extensions like .com) for the following business description:

Bicycle-powered smoothie cart for events. [/INST] pedalpour
crankcup
blendbike
wheelwhisk
spinSip</s>


Example 2:
<s>[INST] Generate 5 creative domain name(s) (without TLD extensions like .com) for the following business description:

Neighborhood bread share and bake sale. [/INST] loaflist
crumbcart
bakebarter
ovenout
sliceSwap</s>


In [7]:
# Define LoRA configuration
lora_config = LoraConfig(
    r=128,  # Rank of the update matrices - those would need doubel check and/or hyperparams optim
    lora_alpha=256,  # Scaling factor
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],

    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)


In [8]:
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=2
)

In [9]:
sft_config = SFTConfig(
    output_dir=f"../models/model_{version}",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=5e-6,
    num_train_epochs=10,
    max_steps=-1,
    logging_steps=10,
    save_steps=100,
    save_total_limit=3,
    warmup_steps=50,
    lr_scheduler_type="cosine",
    report_to=None,
    remove_unused_columns=False,
    max_length=1024,

    # Fix for MPS device - disable fp16, optionally enable bf16
    fp16=False if torch.backends.mps.is_available() else True,
    bf16=True if torch.backends.mps.is_available() else False,

    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [10]:
# Create the SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    peft_config=lora_config,
    args=sft_config,
    callbacks=[early_stopping_callback],
)

print("SFTTrainer initialized successfully!")


Truncating eval dataset: 100%|██████████| 20/20 [00:00<00:00, 15551.74 examples/s]

SFTTrainer initialized successfully!


In [11]:
trainer.train()

/Users/ohm/Projects/sonia/domain.yours/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
1,2.181600,1.994485,1.428482,10848.000000,0.699997
2,1.351200,1.473255,1.448609,21696.000000,0.704030
3,0.813400,1.422758,1.062826,32544.000000,0.723018
4,0.622000,1.603639,0.969260,43392.000000,0.707712
5,0.536300,1.729004,0.856958,54240.000000,0.713209


/Users/ohm/Projects/sonia/domain.yours/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/ohm/Projects/sonia/domain.yours/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/ohm/Projects/sonia/domain.yours/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/ohm/Projects/sonia/domain.yours/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used

TrainOutput(global_step=225, training_loss=1.3642025926378039, metrics={'train_runtime': 431.5364, 'train_samples_per_second': 4.171, 'train_steps_per_second': 1.043, 'total_flos': 2424316510863360.0, 'train_loss': 1.3642025926378039, 'epoch': 5.0})

In [12]:
save_path = f"../models/model_{version}"
trainer.save_model(save_path)
print(f"Model saved to: {save_path}")

tokenizer.save_pretrained(save_path)
print("Tokenizer saved as well")


Model saved to: ../models/model_v1
Tokenizer saved as well


In [13]:
description = "Artisanal coffee roastery with single-origin beans"
prompt = DOMAIN_GENERATION_PROMPT.format(description=description)

test_input = f"<s>[INST] {prompt} [/INST]"

inputs = tokenizer(test_input, return_tensors="pt").to(model.device)
input_token_length = inputs.input_ids.shape[1]

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )


# slicing the output tensor
generated_token_ids = outputs[0, input_token_length:]

# Decode only the new tokens to get the clean response
response = tokenizer.decode(generated_token_ids, skip_special_tokens=True)

print(f"Generated response:\n{response}")

Generated response:
beanbloom
roastriver
groundgrove
brewbarn
cupcrafty
